In [ ]:
import os, sys
import pandas as pd
sys.path.append(os.path.abspath(".."))

GAMES = "../data/raw/games_2025_regular.csv"
FEATS = "../features/team_week_features_2024.parquet"
SEASON = 2025

In [ ]:
games = pd.read_csv(GAMES)
games = games.rename(columns={
    "homeTeam":"home", "awayTeam":"away",
    "homePoints":"home_pts", "awayPoints":"away_pts",
    "neutralSite":"neutral", "seasonType":"season_type"
})
g = games[(games["season"]==2025) & (games["season_type"]=="regular") & (games["completed"]==True)]
g = g[["home","away","week","neutral","home_pts","away_pts"]].copy()

# Check ordering
print("Games rows unsorted where week decreases within same team:")
mask = g.sort_values(["home","week"]).groupby("home")["week"].diff().lt(0)
print(g.loc[mask, ["home","week"]])

Games rows unsorted where week decreases within same team:
Empty DataFrame
Columns: [home, week]
Index: []


In [10]:
from src.datasets import prepare_training_set
games = pd.read_csv(GAMES)
feats = pd.read_parquet(FEATS)

print("Unique weeks in games:", sorted(games['week'].unique())[:15])
print("Unique weeks in feats:", sorted(feats['week'].unique())[:15])
from src.train_predict import run_training_pipeline, run_prediction_pipeline

# Train on all completed 2024 regular-season games, validate on last 2 completed weeks
models = run_training_pipeline(GAMES, FEATS, SEASON, val_weeks=2)


Unique weeks in games: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15)]
Unique weeks in feats: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15)]
[LightGBM] [Info] Number of positive: 2124, number of negative: 1580
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000165 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1017
[LightGBM] [Info] Number of data points in the train set: 3704, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.573434 -> initscore=0.295876
[LightGBM] [Info] Start training from score 0.295876
Validation (weeks [np.int64(15), np.int64(16)]) — LogLoss: 0.6

In [ ]:
unplayed = games[
    (games["season"] == 2025) &
    (games["seasonType"] == "regular") &
    (games["completed"] == False)
][["week", "homeTeam", "awayTeam"]]

print(unplayed)


      week   homeTeam          awayTeam
1272     5  App State           Liberty
2642    10     Taylor  Defiance College


In [12]:
preds = run_prediction_pipeline(models, GAMES, FEATS, SEASON, weeks=[5,10], out_stub="predictions_remaining")
print(preds)


✅ Saved predictions to C:\Users\palle\source\repos\cfp-predictor\data\processed\predictions_remaining_weeks_5-10.csv
        home              away  week  neutral  p_home_win
0  App State           Liberty     5        0    0.537016
1     Taylor  Defiance College    10        1    0.492248


In [ ]:
games = pd.read_csv(GAMES)
mask = (games["season"]==2025) & (games["seasonType"]=="regular") & (games["completed"]==False)
print(games.loc[mask, ["week","homeTeam","awayTeam"]])


      week   homeTeam          awayTeam
1272     5  App State           Liberty
2642    10     Taylor  Defiance College


In [14]:
# Predict the last 3 regular-season weeks (adjust to whatever's left: e.g., 11–13)
preds = run_prediction_pipeline(models, GAMES, FEATS, SEASON, weeks=[11,12,13], out_stub="predictions_regular")
preds.head(10)


ValueError: Found array with 0 sample(s) (shape=(0, 7)) while a minimum of 1 is required by LogisticRegression.